# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb5867b2760>
├── 'a' --> tensor([[-0.9495, -0.9121,  0.2840],
│                   [ 0.3466, -2.0370,  0.9236]])
└── 'x' --> <FastTreeValue 0x7fb5867b2460>
    └── 'c' --> tensor([[-0.7088, -1.3532, -0.1619, -0.0424],
                        [-0.2551,  0.1596, -1.1913,  1.3623],
                        [ 0.2312,  0.4265, -1.7423,  1.2478]])

In [4]:
t.a

tensor([[-0.9495, -0.9121,  0.2840],
        [ 0.3466, -2.0370,  0.9236]])

In [5]:
%timeit t.a

63.2 ns ± 0.0461 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb5867b2760>
├── 'a' --> tensor([[-0.4619, -1.0853, -0.7348],
│                   [ 0.5633, -0.3093,  1.5866]])
└── 'x' --> <FastTreeValue 0x7fb5867b2460>
    └── 'c' --> tensor([[-0.7088, -1.3532, -0.1619, -0.0424],
                        [-0.2551,  0.1596, -1.1913,  1.3623],
                        [ 0.2312,  0.4265, -1.7423,  1.2478]])

In [7]:
%timeit t.a = new_value

64 ns ± 0.0282 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.9495, -0.9121,  0.2840],
               [ 0.3466, -2.0370,  0.9236]]),
    x: Batch(
           c: tensor([[-0.7088, -1.3532, -0.1619, -0.0424],
                      [-0.2551,  0.1596, -1.1913,  1.3623],
                      [ 0.2312,  0.4265, -1.7423,  1.2478]]),
       ),
)

In [10]:
b.a

tensor([[-0.9495, -0.9121,  0.2840],
        [ 0.3466, -2.0370,  0.9236]])

In [11]:
%timeit b.a

57.9 ns ± 0.0428 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.2116,  0.7842,  0.7885],
               [-0.4679, -0.6494,  0.6595]]),
    x: Batch(
           c: tensor([[-0.7088, -1.3532, -0.1619, -0.0424],
                      [-0.2551,  0.1596, -1.1913,  1.3623],
                      [ 0.2312,  0.4265, -1.7423,  1.2478]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.877 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

831 ns ± 0.165 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 10 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 769 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 446 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fb4cc9eefd0>
├── 'a' --> tensor([[[-0.9495, -0.9121,  0.2840],
│                    [ 0.3466, -2.0370,  0.9236]],
│           
│                   [[-0.9495, -0.9121,  0.2840],
│                    [ 0.3466, -2.0370,  0.9236]],
│           
│                   [[-0.9495, -0.9121,  0.2840],
│                    [ 0.3466, -2.0370,  0.9236]],
│           
│                   [[-0.9495, -0.9121,  0.2840],
│                    [ 0.3466, -2.0370,  0.9236]],
│           
│                   [[-0.9495, -0.9121,  0.2840],
│                    [ 0.3466, -2.0370,  0.9236]],
│           
│                   [[-0.9495, -0.9121,  0.2840],
│                    [ 0.3466, -2.0370,  0.9236]],
│           
│                   [[-0.9495, -0.9121,  0.2840],
│                    [ 0.3466, -2.0370,  0.9236]],
│           
│                   [[-0.9495, -0.9121,  0.2840],
│                    [ 0.3466, -2.0370,  0.9236]]])
└── 'x' --> <FastTreeValue 0x7fb4cc9eefa0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 108 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb4cc9cd1c0>
├── 'a' --> tensor([[-0.9495, -0.9121,  0.2840],
│                   [ 0.3466, -2.0370,  0.9236],
│                   [-0.9495, -0.9121,  0.2840],
│                   [ 0.3466, -2.0370,  0.9236],
│                   [-0.9495, -0.9121,  0.2840],
│                   [ 0.3466, -2.0370,  0.9236],
│                   [-0.9495, -0.9121,  0.2840],
│                   [ 0.3466, -2.0370,  0.9236],
│                   [-0.9495, -0.9121,  0.2840],
│                   [ 0.3466, -2.0370,  0.9236],
│                   [-0.9495, -0.9121,  0.2840],
│                   [ 0.3466, -2.0370,  0.9236],
│                   [-0.9495, -0.9121,  0.2840],
│                   [ 0.3466, -2.0370,  0.9236],
│                   [-0.9495, -0.9121,  0.2840],
│                   [ 0.3466, -2.0370,  0.9236]])
└── 'x' --> <FastTreeValue 0x7fb4cc9cd2b0>
    └── 'c' --> tensor([[-0.7088, -1.3532, -0.1619, -0.0424],
                        [-0.2551,  0.1596, -1.1913,  1.3623],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 35.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.1 µs ± 57.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.7088, -1.3532, -0.1619, -0.0424],
                       [-0.2551,  0.1596, -1.1913,  1.3623],
                       [ 0.2312,  0.4265, -1.7423,  1.2478]],
              
                      [[-0.7088, -1.3532, -0.1619, -0.0424],
                       [-0.2551,  0.1596, -1.1913,  1.3623],
                       [ 0.2312,  0.4265, -1.7423,  1.2478]],
              
                      [[-0.7088, -1.3532, -0.1619, -0.0424],
                       [-0.2551,  0.1596, -1.1913,  1.3623],
                       [ 0.2312,  0.4265, -1.7423,  1.2478]],
              
                      [[-0.7088, -1.3532, -0.1619, -0.0424],
                       [-0.2551,  0.1596, -1.1913,  1.3623],
                       [ 0.2312,  0.4265, -1.7423,  1.2478]],
              
                      [[-0.7088, -1.3532, -0.1619, -0.0424],
                       [-0.2551,  0.1596, -1.1913,  1.3623],
                       [ 0.2312,  0.4265, -1.7423,  1.2478]],

In [26]:
%timeit Batch.stack(batches)

77.6 µs ± 89 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.7088, -1.3532, -0.1619, -0.0424],
                      [-0.2551,  0.1596, -1.1913,  1.3623],
                      [ 0.2312,  0.4265, -1.7423,  1.2478],
                      [-0.7088, -1.3532, -0.1619, -0.0424],
                      [-0.2551,  0.1596, -1.1913,  1.3623],
                      [ 0.2312,  0.4265, -1.7423,  1.2478],
                      [-0.7088, -1.3532, -0.1619, -0.0424],
                      [-0.2551,  0.1596, -1.1913,  1.3623],
                      [ 0.2312,  0.4265, -1.7423,  1.2478],
                      [-0.7088, -1.3532, -0.1619, -0.0424],
                      [-0.2551,  0.1596, -1.1913,  1.3623],
                      [ 0.2312,  0.4265, -1.7423,  1.2478],
                      [-0.7088, -1.3532, -0.1619, -0.0424],
                      [-0.2551,  0.1596, -1.1913,  1.3623],
                      [ 0.2312,  0.4265, -1.7423,  1.2478],
                      [-0.7088, -1.3532, -0.1619, -0.0424],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 251 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

303 µs ± 1.71 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
